- 基本流程
- 特征构造
- 构造"id",重复样本及rank特征,上分背后的原因

In [1]:
import pandas as pd

import numpy as np
import lightgbm as lgb
import xgboost as xgb
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score as auc
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error as mae
# from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.model_selection import StratifiedKFold, KFold

from matplotlib.pyplot import plot, show
from IPython.display import display

from tqdm import tqdm

pd.set_option('max_rows', 500) #显示最多行数，超出该数以省略号表示
pd.set_option('max_columns', 100)

d:\Anaconda3\lib\site-packages\dask\config.py:131: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
d:\Anaconda3\lib\site-packages\dask\dataframe\utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
d:\Anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
MODEL_TYPE = 'lgb'

LABEL = 'target'

df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
df_train.shape, df_test.shape

((620356, 13), (206785, 12))

In [3]:
print(df_train.shape, 
      df_test.shape, 
      df_train.drop(columns=['target']).drop_duplicates(subset=[f for f in df_train.columns if f not in ['target', 'uuid', 'common_ts']]).shape,
      df_test.drop_duplicates(subset=[f for f in df_test.columns if f not in ['uuid', 'common_ts']]).shape,
      pd.concat([df_train.drop(columns=['target']), df_test]).drop_duplicates(subset=[f for f in df_test.columns if f not in ['uuid', 'common_ts']]).shape
     )

(620356, 13) (206785, 12) (219956, 12) (98277, 12) (270080, 12)


In [4]:
df_train['udmap'][0], eval(df_train['udmap'][0])

('{"key3":"67804","key2":"650"}', {'key3': '67804', 'key2': '650'})

## 去重

In [5]:
df_train = df_train.drop_duplicates(subset=[f for f in df_train if f not in ['uuid']])
df_train.shape

(617688, 13)

In [6]:
df_train['target'].sum() / df_train['target'].shape[0]

0.1405321132999184

In [7]:
df = pd.concat([df_train, df_test])

In [8]:
a = []
for v in tqdm(df['udmap'].values):
    if v != 'unknown':
        a.extend(list(eval(v).keys()))
keys = sorted(list(set(a)))
keys

100%|██████████████████████████████████████████████████████████████████████| 824473/824473 [00:03<00:00, 248886.08it/s]


['key1', 'key2', 'key3', 'key4', 'key5', 'key6', 'key7', 'key8', 'key9']

In [9]:
keys_df = []
for v in tqdm(df['udmap'].values):
#     print(v)
    if v == 'unknown':
        keys_df.append([-1]*9)
    else:
        v = eval(v)
        this_key = []
        for key in keys:
            if key in v.keys():
                this_key.append(int(v[key]))
            else:
                this_key.append(-1)
        keys_df.append(this_key)
keys_df = pd.DataFrame(keys_df, columns=keys)

100%|██████████████████████████████████████████████████████████████████████| 824473/824473 [00:04<00:00, 181492.61it/s]


In [10]:
keys_df

,key1,key2,key3,key4,key5,key6,key7,key8,key9
0,-1,650,67804,-1,-1,-1,-1,-1,-1
1,-1,484,67804,-1,-1,-1,-1,-1,-1
2,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,-1,650,67804,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...
824468,-1,-1,-1,-1,-1,-1,-1,-1,-1
824469,-1,-1,-1,-1,-1,-1,-1,-1,-1
824470,-1,-1,87725,-1,-1,-1,-1,-1,-1
824471,-1,-1,65765,-1,-1,-1,-1,-1,-1


In [11]:
df = pd.concat([df.reset_index(drop=True), keys_df], axis=1).reset_index(drop=True)

In [12]:
for x in range(1, 9):
    print('-------------', 'x%d'%x)
    display(df['x%d'%x].value_counts())
    

------------- x1


4    506676
0    164827
1    127239
2     25143
3       588
Name: x1, dtype: int64

------------- x2


0    362161
1    180126
3    166753
2    115433
Name: x2, dtype: int64

------------- x3


...

4    522353
1    301551
3       466
2        78
0        25
Name: x6, dtype: int64

------------- x7


7    246223
6    172977
2    119047
8    102791
9     98622
0     51958
4     21521
3      8474
5      2843
1        17
Name: x7, dtype: int64

------------- x8


1    705426
0    119047
Name: x8, dtype: int64

In [13]:
for x in range(1, 10):
    print('key%d'%x, df[df['key%d'%x]==-1].shape[0] / df.shape[0])
#     df[~df.isna()].shape[0]

key1 0.9183769510948205
key2 0.6007874120802016
key3 0.5455691089944729
key4 0.8680684509983954
key5 0.8680684509983954
key6 0.9926377213080355
key7 0.9988744325162862
key8 0.9988501745963785
key9 0.9975693564252559


## 时间戳转换-交叉

In [14]:
df['tm'] = pd.to_datetime(df['common_ts'],unit='ms')
df['day'] = df['tm'].dt.day
df['hour'] = df['tm'].dt.hour
df['dayofweek'] = df['tm'].dt.dayofweek

df['x5_x7'] = df['x5'].astype(str) + '_' + df['x7'].astype(str)
df['x5_x4'] = df['x5'].astype(str) + '_' + df['x4'].astype(str)
df['x7_x4'] = df['x7'].astype(str) + '_' + df['x4'].astype(str)
# 笛卡尔积

df['hour_x7'] = df['hour'].astype(str) + '_' + df['x7'].astype(str)
df['hour_x7_x5'] = df['hour_x7'] + '_' + df['x5'].astype(str)
df['x_all'] = df['x1'].astype(str) + '_'+ df['x2'].astype(str) + '_' + df['x3'].astype(str) + '_' + df['x4'].astype(str)+ '_' + df['x5'].astype(str)+ '_' + df['x6'].astype(str)+ '_' + df['x7'].astype(str)+ '_' + df['x8'].astype(str)

df['hour_x_all'] = df['hour'].astype(str) + '_' + df['x_all']

for f in ['x5_x7', 'x5_x4', 'x7_x4', 'hour_x7', 'hour_x7_x5', 'x_all', 'hour_x_all',]:
    df[f] = df[f].factorize()[0]

## rank特征(统计)

In [17]:
df = df.reset_index()

In [22]:
df['x_all'].value_counts()

284      3662
2        3533
148      2979
66       2798
121      2629
         ... 
14840       1
14841       1
14842       1
11323       1
16786       1
Name: x_all, Length: 16787, dtype: int64

In [31]:
df[df['x_all']==66].sort_values(by='common_ts')[['common_ts', 'target']].dropna()[:100]

,common_ts,target
451925,1688695619410,0.0
393383,1688695619411,0.0
352432,1688695631063,0.0
555284,1688695632182,0.0
330296,1688695632183,0.0
542654,1688695632878,0.0
139030,1688695644375,0.0
165639,1688695649672,0.0
410429,1688695649723,0.0
100463,1688695649723,0.0


In [14]:
df = df.sort_values(by=['common_ts'])
for f in ['eid', 'x4', 'x5', 'x7', 'hour', 'hour_x7_x5', 'hour_x7', 'x5_x7', 'x_all', 'hour_x_all',
         ]:
    df[f+'_sort_ts_rank'] = df.groupby(f)['common_ts'].rank()
df = df.sort_values(by=['index'])

In [15]:
for f in ['eid', 'x4', 'x5', 'x7', 'hour', 'hour_x7_x5', 'hour_x7', 'x5_x7', 'x_all', 'hour_x_all',]:
    df[f+'_ts_std'] = df.groupby([f])['common_ts'].transform('std')
    df[f+'_ts_skew'] = df.groupby([f])['common_ts'].transform('skew')

In [13]:
df_train = df[df[LABEL].notna()].reset_index(drop=True)
df_test = df[df[LABEL].isna()].reset_index(drop=True)

In [23]:
df_train[df_train['x_all']==4].sort_values(by=['common_ts'])['target']

10937     1.0
597336    1.0
147974    1.0
507343    1.0
250845    1.0
264401    1.0
511937    1.0
479811    1.0
402189    1.0
353463    0.0
145992    0.0
447237    0.0
489974    0.0
391110    0.0
31529     0.0
382601    0.0
81433     0.0
399164    0.0
296873    0.0
98166     0.0
402014    0.0
33516     0.0
490211    0.0
4         0.0
100676    0.0
340472    0.0
264973    0.0
Name: target, dtype: float64

In [14]:
feats = [f for f in df_test if f not in [LABEL, 'udmap', 'uuid',  'key4', 'key5', 'key6', 'key7', 'key8', 'key9', 'tm', 'index']]

In [15]:
print(df_train[feats].shape, df_test[feats].shape)
params_lgb = {
    'learning_rate': 0.1,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 64,
    'verbose': -1,
    'seed': 2222,
    'n_jobs': -1,

    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    # 'min_child_weight': 10,
}

params_xgb = {
    'objective': 'binary:logistic',
    'tree_method': 'gpu_hist',
    'eval_metric': 'auc',
    'eta': 0.1,
    'max_depth': 6,
    'subsample':0.8,
    'colsample_bytree': 0.9,
    'seed': 2222
}

fold_num = 5
seeds = [2222]
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()

for seed in seeds:
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    # kf = KFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[LABEL])):
        print('-----------', fold)
        if MODEL_TYPE == 'lgb':
            train = lgb.Dataset(df_train.loc[train_idx, feats],
                                df_train.loc[train_idx, LABEL])
            val = lgb.Dataset(df_train.loc[val_idx, feats],
                              df_train.loc[val_idx, LABEL])
            model = lgb.train(params_lgb, train, valid_sets=[val], num_boost_round=20000, early_stopping_rounds=100, verbose_eval=1000,)
                              # callbacks=[lgb.early_stopping(100), lgb.log_evaluation(1000)])

            oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
            pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats]) / len(seeds)
            importance += model.feature_importance(importance_type='gain') / fold_num

        elif MODEL_TYPE == 'xgb':
            train = xgb.DMatrix(df_train.loc[train_idx, feats],
                                df_train.loc[train_idx, LABEL])
            val = xgb.DMatrix(df_train.loc[val_idx, feats],
                              df_train.loc[val_idx, LABEL])
            watchlist = [(train, 'train'), (val, 'eval')]
            model = xgb.train(params_xgb, dtrain=train, evals=watchlist, num_boost_round=20000,
                              early_stopping_rounds=100, maximize=True,
                              verbose_eval=1000)

            oof[val_idx] += model.predict(xgb.DMatrix(df_train.loc[val_idx, feats])) / len(seeds)
            pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(xgb.DMatrix(df_test[feats]))
            d = model.get_score(importance_type='total_gain')
            d = np.array(list(d.values()))
            importance += np.pad(d, (0, len(feats) - len(d)))

feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
print(feats_importance.sort_values('importance', ascending=False)[:30])

df_train['oof'] = oof

best_score = 0
best_thre = 0
for thre in np.arange(0.1, 0.5, 0.05):
    score = f1_score(df_train[LABEL], (df_train['oof']>thre).astype(int))
    if score > best_score:
        best_score = score
        best_thre = thre
        print(best_thre, best_score)
print(best_score)

(617688, 13) (206785, 13)
----------- 0


d:\Anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
d:\Anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


Training until validation scores don't improve for 100 rounds
[1000]	valid_0's auc: 0.937413
[2000]	valid_0's auc: 0.952089
[3000]	valid_0's auc: 0.958279
[4000]	valid_0's auc: 0.96116
[5000]	valid_0's auc: 0.962616
[6000]	valid_0's auc: 0.963571
[7000]	valid_0's auc: 0.964099
Early stopping, best iteration is:
[7291]	valid_0's auc: 0.964224
----------- 1
Training until validation scores don't improve for 100 rounds
[1000]	valid_0's auc: 0.937977
[2000]	valid_0's auc: 0.951024
[3000]	valid_0's auc: 0.957183
[4000]	valid_0's auc: 0.959751
[5000]	valid_0's auc: 0.9611
Early stopping, best iteration is:
[5418]	valid_0's auc: 0.961689
----------- 2
Training until validation scores don't improve for 100 rounds
[1000]	valid_0's auc: 0.936944
[2000]	valid_0's auc: 0.950787
[3000]	valid_0's auc: 0.95716
[4000]	valid_0's auc: 0.96011
[5000]	valid_0's auc: 0.961481
Early stopping, best iteration is:
[5823]	valid_0's auc: 0.962348
----------- 3
Training until validation scores don't improve for 1

In [ ]:
# (617688, 53) (206785, 53)
# ----------- 0
# d:\Anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
#   _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
# d:\Anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
#   _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
# Training until validation scores don't improve for 100 rounds
# [1000]	valid_0's auc: 0.975696
# [2000]	valid_0's auc: 0.982098
# [3000]	valid_0's auc: 0.983663
# [4000]	valid_0's auc: 0.984294
# Early stopping, best iteration is:
# [4542]	valid_0's auc: 0.984432
# ----------- 1
# Training until validation scores don't improve for 100 rounds
# [1000]	valid_0's auc: 0.974368
# [2000]	valid_0's auc: 0.980596
# [3000]	valid_0's auc: 0.982185
# Early stopping, best iteration is:
# [3289]	valid_0's auc: 0.982456
# ----------- 2
# Training until validation scores don't improve for 100 rounds
# [1000]	valid_0's auc: 0.97453
# [2000]	valid_0's auc: 0.980494
# [3000]	valid_0's auc: 0.98226
# [4000]	valid_0's auc: 0.98298
# Early stopping, best iteration is:
# [4037]	valid_0's auc: 0.983001
# ----------- 3
# Training until validation scores don't improve for 100 rounds
# [1000]	valid_0's auc: 0.975677
# [2000]	valid_0's auc: 0.981781
# [3000]	valid_0's auc: 0.983286
# Early stopping, best iteration is:
# [3774]	valid_0's auc: 0.983653
# ----------- 4
# Training until validation scores don't improve for 100 rounds
# [1000]	valid_0's auc: 0.975064
# [2000]	valid_0's auc: 0.981505
# [3000]	valid_0's auc: 0.983234
# [4000]	valid_0's auc: 0.983782
# Early stopping, best iteration is:
# [3960]	valid_0's auc: 0.9838
#                        name    importance
# 34              eid_ts_skew  93153.911417
# 52       hour_x_all_ts_skew  81273.017014
# 51        hour_x_all_ts_std  75164.677705
# 33               eid_ts_std  73207.213428
# 22               hour_x_all  65931.350918
# 50            x_all_ts_skew  65027.528705
# 49             x_all_ts_std  63886.986879
# 1                 common_ts  62560.596397
# 26          x7_sort_ts_rank  59706.186481
# 24          x4_sort_ts_rank  57060.310923
# 21                    x_all  54196.977088
# 32  hour_x_all_sort_ts_rank  54169.391934
# 27        hour_sort_ts_rank  52930.691247
# 31       x_all_sort_ts_rank  52377.495661
# 29     hour_x7_sort_ts_rank  51379.724489
# 44       hour_x7_x5_ts_skew  50686.233981
# 25          x5_sort_ts_rank  45908.781051
# 20               hour_x7_x5  43695.571866
# 43        hour_x7_x5_ts_std  43529.325864
# 8                        x7  43429.990503
# 30       x5_x7_sort_ts_rank  42704.392993
# 28  hour_x7_x5_sort_ts_rank  39118.217463
# 48            x5_x7_ts_skew  34487.354775
# 40               x7_ts_skew  32173.208515
# 16                    x5_x7  30865.711155
# 47             x5_x7_ts_std  30120.981897
# 41              hour_ts_std  30100.112833
# 37                x5_ts_std  30025.881714
# 18                    x7_x4  29794.721170
# 45           hour_x7_ts_std  29469.130217
# 0.1 0.8156959020426382
# 0.15000000000000002 0.845904952325669
# 0.20000000000000004 0.8621264774952168
# 0.25000000000000006 0.8703134392443107
# 0.30000000000000004 0.8756299674723632
# 0.3500000000000001 0.8781466704664197
# 0.8781466704664197

In [24]:
df_test[LABEL] = pred_y.mean(axis=1).values
df_test[LABEL] = (df_test[LABEL] > thre).astype(int)
df_test[['uuid', LABEL]].to_csv(time.strftime('ans/lgb_%Y%m%d %H%M')+'_%.5f.csv'%best_score, index=False)

In [42]:
ts_df = df_train[df_train['common_ts'].isin(df_test['common_ts'].unique())].groupby('common_ts')['target'].mean().reset_index()
ts_df['target'] = (ts_df['target']>0.4).astype(int)
ts_df = df_train.groupby('common_ts')['target'].mean().reset_index()

In [33]:
ans = pd.read_csv('ans/lgb_20230821 2034_0.87815.csv')
df_test['target_old'] = ans['target']
tmp = df_test[['uuid', 'common_ts', 'target_old']].merge(ts_df, how='left', on='common_ts')
tmp.loc[tmp['target'].isna(), 'target'] = tmp.loc[tmp['target'].isna(), 'target_old'].values
tmp['target'] = tmp['target'].astype(int)
tmp[['uuid', 'target']].to_csv('ans/lgb_20230821 2034_0.87815_post.csv', index=False)

In [ ]:
# 嵌入
# gensim => word2vec 